In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import csv
import pandas as pd
import json
import time
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import string 
import unicodedata as ud
from greek_stemmer import GreekStemmer
import pymongo
import numpy as np
import re
import networkx as nx
import indexer as ind

### Single Document Keyword Extraction

In [69]:
def create_graph_of_words(words_in_row:list)->nx.DiGraph:
    g = nx.DiGraph()
    #add unique words as nodes of the graph
    #g.add_nodes_from(set(words_in_row))
    for j,word in enumerate(words_in_row):
        #add word as a node if it doesnt exist
        if word not in g.nodes:
            g.add_node(word)    
        #generate list of indexes
        gen = (x for x in range(j+1,j+4) if x<len(words_in_row))
        for k in gen:
            #avoid self-loops
            if(word == words_in_row[k]):
                pass
            #if edge does not exist create it, else increase edge weight
            elif((word, words_in_row[k]) not in g.edges):
                g.add_edge(word, words_in_row[k])
                g[word][words_in_row[k]]['weight'] = 1
            else:
                g[word][words_in_row[k]]['weight'] += 1
    return g
def generate_graphs(total_documents:int, chunksize:int, database_collection):
    chunk = []
    counter = 0
    ticks = [x for x in range(0,total_documents,chunksize)]
    ticks.append(total_documents)
    graphs=[]
    with open('stopwords.txt', encoding='utf-8') as file:
        stopwords = [line.rstrip() for line in file]
    for j in range(len(ticks)-1):
        tokens = {}
        #read documents from MongoDB in chunks
        chunk = list(database_collection.find({ }, { "_id": 1, "speech": 1 })[ticks[j]:ticks[j+1]])
        print("Length of chunk: ", len(chunk))
        size_distribution = []
        #chunk = ["This is a sentance","This is another one"]
        #for each speech
        for i, row in enumerate(chunk):
            words_in_row = ind.preprocess_doc(row["speech"], stopwords)
            graph = create_graph_of_words(words_in_row)
            graphs.append(graph)
            #extract keywords and insert to mongo
   
        
        print("CHUNK", counter, " FINISHED")
    
    return graphs

def weighted_undirected_k_core(graph:nx.Graph):
    k = 0
    core_numbers = dict([(node,0) for node in graph.nodes])
    degrees = dict(graph.degree(weight='weight'))
    while(len(graph.nodes)>0):
        min_node = get_min_node(degrees)
        #print(min_node)
        if(degrees[min_node]>k):
            core_numbers[min_node] = degrees[min_node]
            k = core_numbers[min_node]
        else:
            core_numbers[min_node] = k
        for neighbor in graph.neighbors(min_node):
            degrees[neighbor] = degrees[neighbor]  - graph.get_edge_data(neighbor, min_node)['weight']
        graph.remove_node(min_node)
        del degrees[min_node]
        
    return core_numbers

def get_min_node(degrees):
    #sort degrees in increasing orders
    degrees = sorted(degrees.items(), key=lambda d: d[1])
    return degrees[0][0]

#returns the keywords for a document
def get_keywords(database_collection, document_id:string)->list:
    with open('stopwords.txt', encoding='utf-8') as file:
        stopwords = [line.rstrip() for line in file]
    tokens = {}
    #read documents from MongoDB in chunks
    document = list(database_collection.find({"_id":document_id}, { "_id": 0, "speech": 1 }))
    size_distribution = []
    #chunk = ["This is a sentance","This is another one"]
    #for each speech
    words_in_row = ind.preprocess_doc(document[0]['speech'], stopwords)
    graph = create_graph_of_words(words_in_row)
    #extract keywords and insert to mongo
    #core_numbers = weighted_undirected_k_core(graph)
    centrality = nx.betweenness_centrality(graph)
    centrality = dict(sorted(centrality.items(), key=lambda item: -item[1]))
    print(centrality)
    words = list(centrality.keys())
    keywords = words[:int(len(words_in_row)/4)]

    return keywords


In [50]:
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
#mongo_client.drop_database("GreekParliamentProceedings")
client = mongo_client["GreekParliamentProceedings"]
index = client["InvertedIndex"]
database = client["Database"]

In [71]:
#get keywords for a single document
keywords = get_keywords(database, "234")
print(keywords)


{'πολιτ': 0.3572644608551125, 'κυβερνης': 0.1670047360514471, 'ειπ': 0.12939892316319912, 'πραγματ': 0.1104672301027447, 'συνασπισμ': 0.10569015486580276, 'νε': 0.09694859568419974, 'δημοκρατ': 0.09586480901636865, 'συγκυβερνης': 0.08738160619723236, 'πασοκ': 0.08246529959489976, 'κρατ': 0.06846865595423295, 'αποφας': 0.06726673326938995, 'μιλ': 0.06438328745227527, 'μινιμουμ': 0.05756231194120392, 'θεμ': 0.05656285993770532, 'παρ': 0.05173578290186787, 'κυβερν': 0.051023707198146466, 'επρεπ': 0.044266165660291734, 'χρον': 0.04302205549830263, 'πανω': 0.0409863777387555, 'ομοψυχ': 0.03803534002301355, 'γνωριζ': 0.03684056018339678, 'διευθυντ': 0.035545237831017935, 'μερ': 0.031476789488799194, 'ακυβερνης': 0.02875234951998579, 'τρεχ': 0.026568206372375475, 'θετ': 0.023719695704989826, 'εσωτερ': 0.023184711850695105, 'στιγμ': 0.022785349833857504, 'εμπιστοσυν': 0.022015630402358898, 'υπαρχ': 0.021533636366399225, 'αρχηγ': 0.018797862644233308, 'μητσοτακ': 0.01792481038767405, 'καταδειχθ

In [ ]:
#test, pls ignore
dataframe1 = list(collection.find({ }, { "_id": 1, "speech": 1 })[:100000])
for i, row in enumerate(dataframe1):
    #print(row["_id"])
    pass

In [ ]:
#print(graphs[0].edges)
import matplotlib.pyplot as plt
 
plt.savefig('graph.png')
# larger figure size
#pos = nx.circular_layout(graphs[2])

plt.figure(3,figsize=(12,12)) 
nx.draw_networkx(graphs[0], with_labels=True,node_size=80,font_size=15)
plt.show()

## Networkx library test

In [ ]:
#networkx library
import networkx as nx
g = nx.Graph()
l = ['1','2','3']
#g.add_nodes_from(l)

for word in l:
    if(word not in list(g.nodes)):
        g.add_node(word)
for e in list(g.nodes):
    print(e)
    
g.add_edge('1','2')
g.add_edge('2','3')
if(('1','2') in g.edges):
    print(' in')
for e in list(g.edges):
    print(e)

In [ ]:

graphs = generate_graphs(100,100,database)
graph = graphs[78]
for node in graph.nodes:
    for neighbor in graph.neighbors(node):
        if(graph.get_edge_data(neighbor, node)['weight']>1):
            print(graph.get_edge_data(neighbor, node)['weight'])

In [ ]:
core_numbers = dict(core_numbers)
max_core = [num for num in core_numbers.values() if(num==2)]
print(len(max_core))

In [ ]:
test = nx.Graph()
test.add_nodes_from([1,2,3,4,5,6])
test.add_edge(1,2)
test[1][2]['weight'] = 2
test.add_edge(1,4)
test[1][4]['weight'] = 4

test.add_edge(2,3)
test[2][3]['weight'] = 3

test.add_edge(3,4)
test[3][4]['weight'] = 5

test.add_edge(3,5)
test[3][5]['weight'] = 7

test.add_edge(4,5)
test[4][5]['weight'] = 6

test.add_edge(5,6)
test[5][6]['weight'] = 8



In [ ]:
k = 0
core_numbers = dict([(node,0) for node in test.nodes])
#print(graph.nodes)
print('sdfdsf.',len(nx.k_core(test).nodes))
degrees = dict(test.degree(weight='weight'))
while(len(test.nodes)>0):
    min_node = get_min_node(degrees)
    #print(min_node)
    if(degrees[min_node]>k):
        core_numbers[min_node] = degrees[min_node]
        k = core_numbers[min_node]
    else:
        core_numbers[min_node] = k
    for neighbor in test.neighbors(min_node):
        print(neighbor)
        degrees[neighbor] = degrees[neighbor]  - test.get_edge_data(min_node, neighbor)['weight']
    test.remove_node(min_node)
    del degrees[min_node]
    
print(sorted(core_numbers.items(), key=lambda d: d[1]))
print(len(core_numbers))